In [1]:
import pandas as pd
import numpy as np
pd.set_option('mode.chained_assignment', None)

In [2]:
path = r'Experiments/MultiplyTemperature/Exp0(0)/Data'
df1 = pd.read_csv(f'{path}/1.csv',index_col=0)
df2 = pd.read_csv(f'{path}/2.csv',index_col=0)
df3 = pd.read_csv(f'{path}/3.csv',index_col=0)
df4 = pd.read_csv(f'{path}/4.csv',index_col=0)

In [3]:
rules= {0:'OK',1:'processor_sweep',2:'value_combine',3:'error'}
col_rules={'Viscosity_mark':'Viscosity_verbose','Temperature_mark':'Temperature_verbose'} 

In [95]:
df1 = df1.rename(columns=col_rules)
df1[['Viscosity_verbose','Temperature_verbose']] = df1[['Viscosity_verbose','Temperature_verbose']].replace(rules)

df2 = df2.rename(columns=col_rules)
df2[['Viscosity_verbose','Temperature_verbose']] = df2[['Viscosity_verbose','Temperature_verbose']].replace(rules)
df2['time'] =df2['time']+ df1['time'].max()+1

df3 = df3.rename(columns=col_rules)
df3[['Viscosity_verbose','Temperature_verbose']] = df3[['Viscosity_verbose','Temperature_verbose']].replace({np.nan:'OK'})
df3['time'] =df3['time']+ df2['time'].max()+1

df4 = df4.rename(columns=col_rules)
df4[['Viscosity_verbose','Temperature_verbose']] = df4[['Viscosity_verbose','Temperature_verbose']].replace({np.nan:'OK'})
df4['time'] =df4['time']+ df3['time'].max()+1
df = pd.concat([df1,df2,df3,df4])

df= df.dropna(subset=['Viscosity','Temperature'])

In [96]:
# plot_rules = {'processor_sweep':2, 'OK':0, 'error':4, 'value_combine':3,
#        'inner_processor_check':1}
# df[['Viscosity_verbose','Temperature_verbose']] = df[['Viscosity_verbose','Temperature_verbose']].replace(plot_rules)
# dfs['time'] = dfs['time'] / 60


In [102]:
temperature_cond = ((13 < df['Temperature']) & (df['Temperature'] < 45))
viscosity_cond = ((40 < df['Viscosity']) & (df['Viscosity'] < 350))

dfs = df[temperature_cond & viscosity_cond]

In [112]:
dfs[['Temperature','Viscosity']].groupby(by='Temperature').median()

,Viscosity
Temperature,
13.80,344.9
13.90,343.4
14.00,337.9
14.10,335.4
14.16,336.4
...,...
40.20,48.0
40.30,47.5
40.40,46.6


In [103]:
T_goupT= dfs.groupby(by='Temperature')['Temperature']
T_groupV=dfs.groupby(by='Temperature')['Viscosity']
def foo(g):
    whisker_width=0
    q1 = g.quantile(0.45)                
    q3 = g.quantile(0.55)
    iqr = q3 - q1
    return (g >= q1 - whisker_width*iqr) & (g <= q3 + whisker_width*iqr)
    # return np.abs((g - g.median()) / iqr) < 2.22
    # return np.abs((g - g.mean()) / g.std(ddof=0)) < 1

error_mask= T_groupV.apply(foo ).droplevel([0]).reset_index(drop=True).to_numpy()

dfs = dfs[error_mask]


# T_goupT2= dfs2.groupby(by='Temperature')['Temperature']
# T_groupV2=dfs2.groupby(by='Temperature')['Viscosity']
# dfs2
dfs

,time,Viscosity,Viscosity_verbose,Temperature,Temperature_verbose
1,1.0,344.9,OK,13.8,OK
2,2.0,344.9,OK,13.8,OK
3,3.0,344.9,OK,13.8,OK
12,12.0,344.9,OK,13.8,OK
14,14.0,344.9,OK,13.8,OK
...,...,...,...,...,...
5920,202450.0,45.9,OK,40.5,OK
5932,202462.0,45.0,OK,40.5,OK
5935,202465.0,45.7,OK,40.5,OK
5936,202466.0,46.0,OK,40.5,OK


# time plots

In [8]:
import matplotlib.pyplot as plt

In [9]:
%matplotlib qt

In [10]:
# dfs = dfs[(dfs['Temperature_verbose']<3) & (dfs ['Viscosity_verbose']<3)]
# dfs= dfs[(104<dfs['time']) & (dfs['time']<117)]

In [11]:
fig,ax_v =plt.subplots()
ax_T = ax_v.twinx()

ax_v.scatter(dfs['time'],dfs['Viscosity'],color='red',marker='.')
ax_v.set_ylabel('Viscosity',color='red')

ax_T.scatter(dfs['time'],dfs['Temperature'],color='blue',marker='.')
ax_T.set_ylabel('Temperature',color='blue')


# df_masked = dfs[(dfs['Temperature_verbose']>=3) | (dfs ['Viscosity_verbose']>=3)]
# ax_v.scatter(df_masked['time'],df_masked['Viscosity'],color='black',marker='s')
# ax_T.scatter(df_masked['time'],df_masked['Temperature'],color='gray',marker='o')

Text(0, 0.5, 'Temperature')

# Temterature plots

In [12]:
import plotly 
import plotly.graph_objects as go
import plotly.express as px

In [13]:
# dfs = dfs[(dfs['Viscosity_verbose']=='OK')&(dfs['Temperature_verbose']=='OK')]

In [114]:
g = dfs[['Temperature','Viscosity']].groupby(by='Temperature').median()

In [113]:
fig = go.Figure()
fig = px.box(
    dfs,
    x="Temperature",
    y="Viscosity",
    color='Viscosity_verbose',
)

fig.add_traces([
    go.Scatter(name='Median',
               x=g['Temperature'],
               y=g['Viscosity'],
               mode='lines'),
    go.Scatter(name='Mean', x=T_goupT.max(), y=T_groupV.mean(), mode='lines'),
    # go.Scatter(
    #     name='Errors',
    #     x=T_goupT2.max(),
    #     y=T_groupV2.mean(),
    #     mode='markers',
    # ),
])
fig.update_layout(xaxis=dict(range=(13.5, 40.5)),
                  yaxis=dict(range=(0, 350)),
                  margin=dict(l=10, r=10, b=10, t=10, pad=0))
fig.show()

NameError: name 'g' is not defined

In [78]:
test= pd.DataFrame([
    {'a':1,'b':2},
    {'a':1,'b':3},
    {'a':1,'b':2},
    {'a':1,'b':10},
    {'a':1,'b':1},
    {'a':2,'b':8},
    {'a':2,'b':9},
    {'a':2,'b':70},
    {'a':2,'b':23},
    {'a':2,'b':9},
    
])

In [85]:
def foo(g):
    whisker_width=0
    q1 = g.quantile(0.01)                
    q3 = g.quantile(0.99)
    iqr = q3 - q1
    return (g >= q1 - whisker_width*iqr) & (g <= q3 + whisker_width*iqr)
    # return np.abs((g - g.median()) / iqr) < 2.22
    # return np.abs((g - g.mean()) / g.std(ddof=0)) < 1
mask = test.groupby(by='a')['b'].apply(foo).droplevel([0]).reset_index(drop=True).to_numpy()
mask

array([ True,  True,  True, False, False, False,  True, False,  True,
        True])

In [86]:
test[~mask]

,a,b
3,1,10
4,1,1
5,2,8
7,2,70
